# Part 1: Can We Talk About the Yelp Users?
Depending on the question we are asking of the Yelp data, we might want to know aspects of user behavior such as:
* What was the rating given on their first review?
* How often do they review?
* What categories (or cuisines within the "Restaurants" category) do they review?
* Do their reviews get voted as funny, useful, or cool?
* How do their ratings of businesses compare to the ratings of other other users regarding those same businesses?
* What businesses do they write tips about?
* Do they write tips about the same businesses they review?
* Do they have reviewing or tip writing patterns as far as the days-of-the-week or months-of-the-year that they write?
* How long are their reviews?
* etc.

To be able to answer these questions, we need to have their reviews.
### The Data We Have:
* All of the reviews for any business in 10 metro areas - about 8 million reviews (other than reviews Yelp filtered out as unreliable) 
* All of the tips (mini-reviews) from those same 10 metro areas - about 1.3 million tips
* Yelp business information on all of the businesses reviewed in those reviews or tips - about 209,000 businesses
* Yelp user info on all of the users who wrote those reviews and tips - about 2 million users

### One question we need to ask ourselves is:
# *Do we have all of the reviews written by those users?*

Let's load the user & review data to find out.  

### Step 1:
**In the next cell:** Load the user data into a DataFrame named `df_userData` and then count the number of records and print the schema. 
If you prviously ran the *Building Review and User Tables* notebook, this should take less than 1 minute, but if you have not,it may take about 9 minutes.

In [0]:
if ( spark.catalog._jcatalog.tableExists("users_table") ):
  print("Loading user data from table ...")
  df_userData = spark.read.format('parquet').table("users_table").cache()
  print("Loaded user data from table")
else:
  print("The Building Review and User Tables notebook has not been run, loading from files ...")
  df_userData = spark.read.json("/yelp/user.bz2").withColumn("friend_count",f.size(f.split(f.col("friends"),'\s*,\s*'))).drop("friends").cache()
  # Does not already exist as a table, so write it out
  df_users.write.mode("overwrite").format('parquet').saveAsTable("users_table")
  print("Loaded user data from file")

print("number of users:", df_userData.count())
df_userData.printSchema()

Loading user data from table ...
Loaded user data from table
number of users: 1968703
root
-- average_stars: double (nullable = true)
-- compliment_cool: long (nullable = true)
-- compliment_cute: long (nullable = true)
-- compliment_funny: long (nullable = true)
-- compliment_hot: long (nullable = true)
-- compliment_list: long (nullable = true)
-- compliment_more: long (nullable = true)
-- compliment_note: long (nullable = true)
-- compliment_photos: long (nullable = true)
-- compliment_plain: long (nullable = true)
-- compliment_profile: long (nullable = true)
-- compliment_writer: long (nullable = true)
-- cool: long (nullable = true)
-- elite: string (nullable = true)
-- fans: long (nullable = true)
-- funny: long (nullable = true)
-- name: string (nullable = true)
-- review_count: long (nullable = true)
-- useful: long (nullable = true)
-- user_id: string (nullable = true)
-- yelping_since: string (nullable = true)
-- friend_count: integer (nullable = true)

### Step 2: What User Data Do We Want?
From the schema, you will see that we have a lot of information, mostly about complements, but we also have some other 
data - such as: the number of reviews they wrote, when they started yelping, and the years they were elite.

**In the next cell:** use the `select` method to create a DataFrame named `df_users` that has the following:
* their user ID
* their name
* the count of the number of reviews they wrote
* their average star rating

Print the record count and show the first few records to profile your data to make sure your transformation is generating what you expected.

In [0]:
df_users = df_userData.select("user_id", "name", "review_count", "average_stars").cache()

print("number of users:", df_users.count())
df_users.show(5,truncate=False)
df_userData.unpersist()

number of users: 1968703
+----------------------+-------+------------+-------------+
user_id |name |review_count|average_stars|
+----------------------+-------+------------+-------------+
Qw_H_0xUHJN1eybBmbhZsA|Ken |4 |4.75 |
E3K2qQ0bkWrSyiikh4OnjA|Brandon|23 |4.61 |
Ba_mM1i4OAuZ57MVYsLSwQ|Rick |1 |5.0 |
GRujMvFN83-IBQAwTkteLQ|B.v. |13 |3.92 |
b4YX1azbMwgV5VLu3lrpiA|Rahul |1 |1.0 |
+----------------------+-------+------------+-------------+
only showing top 5 rows

Out[7]: DataFrame[average_stars: double, compliment_cool: bigint, compliment_cute: bigint, compliment_funny: bigint, compliment_hot: bigint, compliment_list: bigint, compliment_more: bigint, compliment_note: bigint, compliment_photos: bigint, compliment_plain: bigint, compliment_profile: bigint, compliment_writer: bigint, cool: bigint, elite: string, fans: bigint, funny: bigint, name: string, review_count: bigint, useful: bigint, user_id: string, yelping_since: string, friend_count: int]

### Step 3: What Review Data Do We Want?
We will need the user_id to be able to match to the user data, and we may want the review_id just in case we want to look back at the raw data at some point.  Since the stars in the review is raw data, we should bring that along to so we could recalculate the average.  Don't bring the review itself - that takes up a lot of space.  

In the next cell: load the review data into a DataFrame named `df_reviews` **and do a select to get the following fields:**
* user_id
* stars

After you have selected only these 2 fields, also `cache` the resulting DataFrame

Name the DataFrame df_reviews (after doing the `select`) and print a count of the number of reviews and show the schema for your data.

In [0]:
if ( spark.catalog._jcatalog.tableExists("reviews_without_text_table") ):
  print("Loading review data from table ...")
  df_reviewData = spark.read.format('parquet').table("reviews_without_text_table").cache()
  print("loaded review data from table")
else:
  print("The Building Review and User Tables notebook has not been run, loading review data from files ...")
  df_reviewData = spark.read.json("/yelp/review.bz2").select("business_id","user_id","cool","funny","useful","date","stars").cache()
  # Does not already exist as a table, so write it out
  df_reviewData.write.mode("overwrite").format('parquet').saveAsTable("reviews_without_text_table")
  print("loaded review data from file")


df_reviews = df_reviewData.select("user_id", "stars").cache()
print("number of reviews:", df_reviews.count())
df_reviews.printSchema()
df_reviews.show(truncate=False)
df_reviewData.unpersist()

Loading review data from table ...
loaded review data from table
number of reviews: 8021122
root
-- user_id: string (nullable = true)
-- stars: double (nullable = true)

+----------------------+-----+
user_id |stars|
+----------------------+-----+
c4Y3EX5aXYixkyVD6uSMag|4.0 |
fHTAUYCd5HuboQp6HymzjQ|3.0 |
ecXDhY91U-ORKk_U7wwdeA|3.0 |
ZKpETYqLymDFL76hwkUaSQ|1.0 |
zcp389V72h0DKl69X72wKQ|3.0 |
xbksPRSmtsmDNiR1VoTOPA|2.0 |
dG-J0IG122Nwqp0Co21Rkw|1.0 |
O4ADr-eEYkP1OKQUxi49EQ|1.0 |
LeSXKUJeFvX2aSvfPnl7dg|5.0 |
_fX--5YqPXMB4b7q22VWkw|5.0 |
HhfNSpdHGSGiuyi1anEOCQ|4.0 |
YKiRYAlL7SEV5P29ppyl-A|5.0 |
Zt_z6uOk0fPXjsFvCO29AQ|5.0 |
y0HCy6GpDFnsQXhCG0ul-g|5.0 |
SB7Ow6Y52kNcHdecdqvKhQ|5.0 |
0SrL97xFh-mVMOxUFDFgEg|1.0 |
yZ3Zm1fBfqlBWhFikW7MEw|4.0 |
hRzmJCPmV1_ggM5jSxq2bA|5.0 |
HS3y3GltOH_pJ37-EJwOng|4.0 |
R-qaBztGNZoxEbKetmvdfg|5.0 |
+----------------------+-----+
only showing top 20 rows

Out[8]: DataFrame[business_id: string, user_id: string, cool: bigint, funny: bigint, useful: bigint, date: string, stars: double]

### Step 4: Grouping the Reviews by User
We want to know the number of reviews each user has in the dataset (the user data has a count of all of their reviews).  We also want to calculate each user's average rating for the reviews in the dataset.

The following cell uses PySpark to do the grouping.

#### What is this doing:

The `groupBy` method returns an instance of `GroupedData` and the `agg()` method of `GroupedData` allows us to do aggregations (groupings) of our data based on columns. 

* Documentation for `groupBy` can be found <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy" target="_blank">here</a>
* Documentation for `agg` can be found <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.agg" target="_blank">here</a>

The parameters for `agg` is one (or more) name : value pairs where the name is the name of a column in the data, and the value is the type of aggregation we want to do:

* count
* sum
* min
* max
* avg
* stddev

If multiple columns are being aggregated, they are passed as a dictionary of name : value pairs:

`{"user_id": "count", "stars": "avg"}`

Here the columns we are grouping are the `review_id` and `stars` columns.  The aggregations we are doing are `count` (how many reviews are there for the user we are grouping on),  and `avg` (what is the average rating for that user in the review data).

**NOTE**: One limitation of the `agg` method when compared to using aggregate SQL functions in a `spark.sql()` query is that the dictionary passed as a parameter to the `agg` method can only have one aggregation for a field, so while you can aggregate different fields in the same query as we are doing here, you cannot get the average for the stars field and another aggregate such as the minimum, maximum, or standard deviation for the stars field in the same query.

The `withColumnRenamed` method of the DataFrame allows us to specify a new name for a column by providing the old name and the new name.  If we don't
rename our columns, the column names will be the formulas used to create them (and that will make our teammates go bald trying to figure out our subsequent queries).

When doing this in your project, you may need to run the query once to see what the formulas are in the column headings and then add the call to the 
`withColumnRenamed` method and rerun the query to rename the columns.

In [0]:
df_reviews.groupBy("user_id").agg({"user_id": "count", "stars": "avg"}).\
                            withColumnRenamed("count(user_id)", "reviewCount").\
                            withColumnRenamed("avg(stars)", "avgStars").show(truncate=False)

+----------------------+-----------+------------------+
user_id |reviewCount|avgStars |
+----------------------+-----------+------------------+
JwGEWAcWTEFDL23hweF6Aw|1 |5.0 |
rYRXfMa58nL7mhLtvbSOsg|8 |3.5 |
Kbw42VOBrZ_Z9_r0tuEIlA|6 |5.0 |
vwGKWnhZLBM93MvkmTtr9Q|1 |1.0 |
sZdjfhtId7Pz6jvIOjgFtw|2 |4.5 |
5QCdtVsWH2eURvH9MBdVig|10 |3.0 |
oFVHhkVTm4v1KhLHcCJ8ng|10 |1.1 |
rXSTTSDOhcjLrv16kWsM2w|94 |4.372340425531915 |
gu7Y-lwgMwbKxuzkeYvfRw|5 |3.4 |
WQ3nv1TofxE1GIWBJEU9ug|13 |4.076923076923077 |
UH4V3gIzHedK2VehzB5JiQ|1 |1.0 |
MPmProg1hIdZkez1QLnvMQ|1 |5.0 |
Inh6RQ9BjpV05V74OLYV4g|4 |4.0 |
max8d9I77RAETlIH5rzy5g|1 |5.0 |
RO78oDy7vbEcOJU8anY5MQ|113 |4.0265486725663715|
5mi369sSkAmBbEfyoeYx3A|14 |3.4285714285714284|
NzJ_HMCJ6ei-zf5zDerVgw|25 |3.4 |
lNJF8IjkzV4ECLWadNJ1LA|4 |4.75 |
BxNBl3VrzLBAkwpJ3EUgXg|24 |3.5833333333333335|
2T5tAwHv8hSnUCoQXlL-_Q|46 |3.9565217391304346|
+----------------------+-----------+------------------+
only showing top 20 rows

### Step 5a: Creating Temporary Views
In the following cell we create temporary views against the `df_users` and `df_reviews` DataFrames.  Keep in mind that the only place we can use these is within queries run using the `sql` method of our SparkSession (named `spark`).  We use `createOrReplaceTempView` instead of `createTempView`.  By saying to replace, if we rerun our code during a session Spark will recreate the temporary view - which if the underlying DataFrame changed, that's what we want. Otherwise calling create when the temporary view already exists would cause an error.

In [0]:
df_users.createOrReplaceTempView("users")
df_reviews.createOrReplaceTempView("reviews")

### Step 5b: Now let's try the same aggregation using Spark SQL

Previously we used COUNT(column_name) to aggregate records and get a count of the rows.  Following is a list of the 
common aggregations, which ones do we need for this query?

* COUNT(column_name)
* SUM(column_name)
* MIN(column_name)
* MAX(column_name)
* AVG(column_name)
* STDDEV(column_name)

In [0]:
df_userReviews = spark.sql("""
SELECT user_id, COUNT(user_id) AS reviewCount,
       AVG(stars) AS avgStars
FROM reviews
GROUP BY user_id
""").cache()
df_userReviews.show(truncate=False)

+----------------------+-----------+------------------+
user_id |reviewCount|avgStars |
+----------------------+-----------+------------------+
JwGEWAcWTEFDL23hweF6Aw|1 |5.0 |
rYRXfMa58nL7mhLtvbSOsg|8 |3.5 |
Kbw42VOBrZ_Z9_r0tuEIlA|6 |5.0 |
vwGKWnhZLBM93MvkmTtr9Q|1 |1.0 |
sZdjfhtId7Pz6jvIOjgFtw|2 |4.5 |
5QCdtVsWH2eURvH9MBdVig|10 |3.0 |
oFVHhkVTm4v1KhLHcCJ8ng|10 |1.1 |
rXSTTSDOhcjLrv16kWsM2w|94 |4.372340425531915 |
gu7Y-lwgMwbKxuzkeYvfRw|5 |3.4 |
WQ3nv1TofxE1GIWBJEU9ug|13 |4.076923076923077 |
UH4V3gIzHedK2VehzB5JiQ|1 |1.0 |
MPmProg1hIdZkez1QLnvMQ|1 |5.0 |
Inh6RQ9BjpV05V74OLYV4g|4 |4.0 |
max8d9I77RAETlIH5rzy5g|1 |5.0 |
RO78oDy7vbEcOJU8anY5MQ|113 |4.0265486725663715|
5mi369sSkAmBbEfyoeYx3A|14 |3.4285714285714284|
NzJ_HMCJ6ei-zf5zDerVgw|25 |3.4 |
lNJF8IjkzV4ECLWadNJ1LA|4 |4.75 |
BxNBl3VrzLBAkwpJ3EUgXg|24 |3.5833333333333335|
2T5tAwHv8hSnUCoQXlL-_Q|46 |3.9565217391304346|
+----------------------+-----------+------------------+
only showing top 20 rows

### Step 5c: unpersist() the df_reviews DataFrame
Now that we have aggregated the review data (and cached it), we no longer need the `df_reviews` in memory, so `unpersist()` it to free up that memory.

In [0]:
df_reviews.unpersist()

Out[12]: DataFrame[user_id: string, stars: double]

### Step 5d: Creating A new temporary view
In the prior query you did a SQL query against a temporary view named `reviews`, but the result generated was a new DataFrame; not a temporary view.
Since we want to use that result in subsequent queries, we need to create a new temporary view based on the `df_userReviews` DataFrame.

In [0]:
df_userReviews.createOrReplaceTempView("userReviews")

# Part 2
### Step 6: Profiling With JOINS to Confirm Our Understanding of the Data
Our understanding of the data is that the user data contains every user that wrote a review in our data
and that every user in our data has at least one review.  We can do a join between the `userReviews` and 
`users` temporary views to confirm this.

In the following cell, do a left join of the temporary views `users` and `userReviews` based on the user ID.
Since the `users` view is on the left side of the join, 
all of the rows in the `users` view will be included in the result; even if there is no matching user ID in the `userReviews` view.

You may be wondering why we enclosed the query in triple-quotes.  The query itself is a string parameter, so we can use a 
feature in Python where you can have a string be on multiple lines without needing backslashes as line-continuation characters.

A couple things to note about our query:
* In the FROM clause we specify "U" as an alias for the `users` temporary view, so in the SELECT clause we can say `U.user_id`, otherwise if we just said `user_id`, Spark would throw up it's hands and say, "that's ambiguous, which user_id?" since that field is in both views.
* In the join expression `ON U.user_id = R.user_id` there is only one `=` sign instead of the two `=` signs we use in the PySpark
`join` method.
* We aliased (renamed) review count columns from the user and review data as `uCount` and `rCount` respectively.  Since we have two different review counts, this makes it easier to keep track of where the counts came from in subsequent queries.  We also aliased the average ratings from each temporary view.
* We capitalized all of the keywords such as `SELECT`, `FROM`, `FULL OUTER JOIN`, etc.  This is not required, but is a common 
convention when writing SQL to make it more human readable for the next human to read it (which could be you a week later, 
your teammate, co-worker, or a future job interviewer).

#### In the following cell, and the FROM and ORDER BY clauses

In [0]:
# Add the FROM clause
df_usertest = spark.sql("""
SELECT U.user_id, U.name, U.review_count AS uCount, U.average_stars as uAvgStars,
       R.reviewCount AS rCount, R.avgStars AS rAvgStars
FROM 
users as U left join userReviews as R
ON U.user_id = R.user_id
order by uCount desc
""")
df_usertest.show(truncate=False)

+----------------------+--------+------+---------+------+------------------+
user_id |name |uCount|uAvgStars|rCount|rAvgStars |
+----------------------+--------+------+---------+------+------------------+
8k3aO-mPeyhbR5HUucA5aA|Victor |14455 |3.28 |8 |2.625 |
RtGqdDBvvBCjcu5dUqwfzA|Shila |12772 |3.87 |133 |4.037593984962406 |
hWDybu_KvYLSdEFzGrniTw|Bruce |12487 |3.64 |1602 |3.6853932584269664|
Hi10sGSZNxQH3NLyWSZ1oA|Fox |11112 |3.8 |374 |4.021390374331551 |
P5bUL3Engv-2z6kKohB6qQ|Kim |9875 |3.8 |5 |4.4 |
8RcEwGrFIgkt9WQ35E6SnQ|George |7745 |3.49 |102 |3.8137254901960786|
nmdkHL2JKFx55T3nq5VziA|Nijole |7626 |3.71 |198 |3.8838383838383836|
Xwnf20FKuikiHcSpcEbpKQ|Kenneth |6762 |3.32 |391 |3.452685421994885 |
CxDOIDnH8gp9KXzpBHJYXw|Jennifer|6633 |3.33 |4331 |3.24797968136689 |
HFECrzYDpgbS5EmTBtj2zQ|Eric |5500 |3.93 |148 |3.885135135135135 |
kS1MQHYwIfD0462PE61IBw|Rob |5156 |3.82 |65 |3.8923076923076922|
WG3w_73scm_JUWJ_3Lgn0Q|Jack |5013 |3.87 |21 |3.8095238095238093|
m07sy7eLtOjVdZ8oN9JKag|Ed |4913 |3.66 |116 |3.6120689655172415|
Xj0O2l0bp633ebmG468aZw|Andrew |4851 |3.73 |86 |3.3488372093023258|
bQCHF5rn5lMI9c5kEwCaNA|Vincent |4845 |3.77 |167 |3.81437125748503 |
XYSDrIef7g4Gmp3lNFVO6A|Neal |4828 |3.96 |306 |3.9052287581699345|
IucvvxdQXXhjQ4z6Or6Nrw|Sunil |4784 |3.46 |58 |3.586206896551724 |
wZPizeBxMAyOSl0M0zuCjg|Jess |4697 |3.63 |103 |3.825242718446602 |
U4INQZOPSUaj8hMjLlZ3KA|Michael |4631 |3.9 |1762 |3.9687854710556185|
bLbSNkLggFnqwNNzzq-Ijw|Stefany |4627 |3.39 |2555 |3.324853228962818 |
+----------------------+--------+------+---------+------+------------------+
only showing top 20 rows

### Step 7: Hmmmm....
We did a left join, and any users that are not in the summarized user data from the reviews would be null (for the columns from the user data), but we are only seeing 20 rows out of 2 million - the odds of a problem row showing up are low.  In the following cell, we use the PySpark `filter` method to show the records where the `rCount` field is null.

To check if the value in a specific column is null, we use the `isnull()` method.

In [0]:
df_usertest.filter(df_usertest.rCount.isNull()).show() 

+-------+----+------+---------+------+---------+
user_id|name|uCount|uAvgStars|rCount|rAvgStars|
+-------+----+------+---------+------+---------+
+-------+----+------+---------+------+---------+

### Step 8: ANTI JOIN
There is another way we could check if there were records in the `user` view.  Instead of having to create the DataFrame
and then separately run a filter, we could have used a `LEFT ANTI JOIN` instead of a `LEFT OUTER JOIN`.  The anti join would have
only included those rows from the view on the left (`users`) that did ***NOT*** have a matching row in the view on the right (`userReviews`).

**In the next cell:** Create a `LEFT ANTI JOIN` query, but since we already checked for records in `users` that were not in `userReviews`, 
put `userReviews` on the left.  The left anti join does not include fields from the view on the right-hand side, so instead of listing fields 
in the `SELECT` clause, just include the asterisk `*` symbol as a wildcard for all fields.

Since we are not going to need to filter the result, there is no need to save the DataFrame created, you can just show it.

This would be an example of individual value profiling since we are looking for those rows where there is not a matching user record for a review, which would indicate that that review has an invalid user_id.

In [0]:
# Add your query
spark.sql("""
select * from userReviews 
left anti join users 


""").show()

+-------+-----------+--------+
user_id|reviewCount|avgStars|
+-------+-----------+--------+
+-------+-----------+--------+

### Step 9: Whoa!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Some of our users have a ***LOT*** of reviews that are not in the dataset!  Is this a problem? 

**FIRST: (Step 9a)**
Go back up to step 6 and the query that generates the `df_usertest` DataFrame. Add an `ORDER BY` clause to sort on the 
`uCount` field in descending (DESC) order.  This will list our users who have written the most reviews.

We have a user that wrote well over 14K reviews and we have less than 10 of them in the dataset!  Think about the data we have - 
if a user wrote even 1 review in one of the 10 metro areas in our dataset, then their user data is in the dataset, but 
if they wrote 10,000 reviews in New York, Los Angeles, Chicago, or Houston (the 4 largest cities in the United States),
those reviews are not in our dataset.

We need to calculate a dataset percentage - how much of a user's data we have.

**Next: (Step 9b) In the next cell:** Copy the `df_usertest` query from **Step 6** above and modify it as follows:
* Add a new field named `datasetPercentage` that's the ratio of `reviewCount` from `datasetReviews` over 
the `review_count` from the `users` view.
* Since we now know the same users are in each view, use an `INNER JOIN` instead of a `LEFT OUTER JOIN` 
* Order the results in descending order based on the `datasetPercentage` ratio you added.
* Store the result of the query in a DataFrame named `df_datasetPercentage`.
* Show the results of the query.

**Remember the sage advice of MC Hammer**: Break it down!
* Copy your earlier query
* Build your changes incrementally
* Do quick profiling queries to make sure your query seems to work (e.g., count or filter)

In [0]:
# Add your query here
df_datasetPercentage = spark.sql("""


SELECT U.user_id, U.name, U.review_count AS uCount, U.average_stars as uAvgStars,
       R.reviewCount AS rCount, R.avgStars AS rAvgStars, R.reviewCount/ U.review_count as datasetPercentage
FROM 
users as U inner join userReviews as R
ON U.user_id = R.user_id
order by datasetPercentage desc


""")
df_datasetPercentage.show(truncate=False)

+----------------------+--------+------+---------+------+-----------------+-----------------+
user_id |name |uCount|uAvgStars|rCount|rAvgStars |datasetPercentage|
+----------------------+--------+------+---------+------+-----------------+-----------------+
mZDTMGAKWdTVj7lXqHSB6g|Julius |1 |5.0 |11 |5.0 |11.0 |
2wD6U-IlZe9PKZ92j4x1Tg|A N |1 |1.0 |8 |1.0 |8.0 |
C5siJ6bxhxmNe1Yo3Zc5yQ|Grace |1 |5.0 |8 |5.0 |8.0 |
Oq9TjfJ7r117VJpjGg3zDA|Kevin |2 |5.0 |14 |5.0 |7.0 |
XAgLWUIZ7fRxy7gDs23I8g|Patty |1 |1.0 |7 |1.0 |7.0 |
b9r3Og8W1BCrrXwfRdX98w|Diana |1 |5.0 |7 |5.0 |7.0 |
qHiC5C_3uGUC8ndTPccCew|Marie |1 |4.71 |7 |4.714285714285714|7.0 |
w4qPnOhhkxE9L5CXZxh3pg|Linda |4 |4.76 |25 |4.76 |6.25 |
DpAWTFBqOZ9zZfgkuP993Q|Kam |1 |5.0 |6 |5.0 |6.0 |
rwrnM_d4b5pM0OEUlwZ-gw|Adelinde|1 |4.0 |6 |4.0 |6.0 |
P0mxRLUVZHFQZjQn6hveig|Jong |1 |5.0 |6 |5.0 |6.0 |
VKTyHOmQvgit0mFMcxO01A|Brian |1 |1.0 |6 |1.0 |6.0 |
Zm4-RVaQ_8SQ9vGdnAXPAA|Diane |1 |5.0 |6 |5.0 |6.0 |
LOeroXeJXCrwvnYldTJwrg|Li |1 |5.0 |6 |5.0 |6.0 |
HYCcnIgrG5UqT0Lo-Yiipw|Jacob |1 |1.0 |6 |1.0 |6.0 |
z60tKk_Fw6toEbu_s-v7aA|Karen |2 |5.0 |11 |5.0 |5.5 |
Kcq0fy2jeBKp6jQ7EgxLMw|Tara |1 |5.0 |5 |5.0 |5.0 |
esLcIIJifowzvY3zdrhkNQ|Sharron |1 |4.8 |5 |4.8 |5.0 |
eKyPCwvGnsCCE0wt-sEqCQ|Tom |1 |4.0 |5 |4.0 |5.0 |
9FmHN1qeAywnFUcy2jdSYg|Tom |1 |1.0 |5 |1.0 |5.0 |
+----------------------+--------+------+---------+------+-----------------+-----------------+
only showing top 20 rows

### Are there users with NO reviews in the user data?
Above we are looking at users with more reviews in the review data than the user data.  Are there any with no reviews in the user data?
If so, we would be dividing by zero and ending up with null for the `datasetPercentage`.  We can filter for these users.

In [0]:
print("users with a null dataset %:",df_datasetPercentage.filter("datasetPercentage IS NULL").count() )

users with a null dataset %: 18

In [0]:
df_datasetPercentage.filter("datasetPercentage IS NULL").show(truncate=False)

+----------------------+----------+------+---------+------+---------+-----------------+
user_id |name |uCount|uAvgStars|rCount|rAvgStars|datasetPercentage|
+----------------------+----------+------+---------+------+---------+-----------------+
kyFCu-Pq687UI9joS5Klzg|Tim |0 |5.0 |1 |5.0 |null |
SoFv1NZbpnZdL051sVVXBA|Susanna |0 |5.0 |1 |5.0 |null |
XAz5ONnjFxn2aCFvmv0Lnw|Carolyn |0 |1.0 |1 |1.0 |null |
0EoFWrwXLcY0HVmTqa13Bw|Amanda |0 |1.0 |1 |1.0 |null |
2EB7H2HJhsshiCLIbIIopg|Jason |0 |4.0 |1 |4.0 |null |
AbbeFj9SPxOoHNKkeDVn6w|Janet |0 |1.0 |1 |1.0 |null |
7RM6w3OHu95ifgXXfl5aLw|Emma |0 |2.0 |1 |2.0 |null |
uqP22Aaz5BV0-4gcmRR00A|James |0 |1.0 |1 |1.0 |null |
t7kIahAbZ3yUsaUOsJ1DfQ|Abdullah M|0 |1.0 |1 |1.0 |null |
-d8nnc-pp6qj_6qnp4IN-g|Luke |0 |5.0 |1 |5.0 |null |
Gr2P0zi6MoeZj-EFFYKDig|Daniel. |0 |5.0 |1 |5.0 |null |
SA5rx2OI_pZZnaCFfe5q-w|Robert |0 |5.0 |1 |5.0 |null |
oCmUIcQVFD6h_ANjz2VAHA|Beidou |0 |5.0 |1 |5.0 |null |
3Ja_GCwYABtbIyNeAFEaew|Brandon |0 |1.0 |1 |1.0 |null |
3MXn44GeVMQ2DgCzv7-2hg|Celeste |0 |1.0 |1 |1.0 |null |
bPCzwbevfP0wZq89fpYdyg|Madi |0 |5.0 |1 |5.0 |null |
_ccSMiXTeSQXKkugIDu3bg|Kim |0 |5.0 |1 |5.0 |null |
Bae554PB5n38qWIOY9ijpQ|Genevieve |0 |4.0 |1 |4.0 |null |
+----------------------+----------+------+---------+------+---------+-----------------+

# More Profiling
That last query is a head scratcher.  Do we have some over-achievers? There are users with over 100% of their reviews in the dataset.  At this point 
we only see 20 rows, so we don't know if this is a pervasive problem we need to look into.  

### Step 10: How many "Overachievers"?
In the following cell add a query that uses the `COUNT` aggregate function to get a count of the number of users where their `datasetPercentage` is greater 
than 1 (meaning over 100%).  Since the `COUNT` function 
is an aggregate function, we would often add a `GROUP BY` clause to say what fields we want to group the data by, but since we want to group
all of the rows to get a single count (where this condition applies), we don't need a `GROUP BY` clause in this case.

However, since we only want to count the rows where this condition is true (where the `datasetPercentage` is over 100%), we need a `WHERE` clause with that condition.

There is one other addition we needed to make to the following cell.
In the earlier cell when we ran the query to calculate the dataset percentage, the result was assigned to a DataFrame, not a temporary view, so
we need to create a temporary view named `datasetPercentage` from the `df_datasetPercentage` DataFrame.

In [0]:
# Add your query below

df_datasetPercentage.createOrReplaceTempView("datasetPercentage") 

spark.sql("""
select count(user_id) as highPercentage from datasetPercentage
where datasetPercentage>1

""").show()

+--------------+
highPercentage|
+--------------+
 27956|
+--------------+

### Step 11: How Many Users Do We Know a Lot About?

**Add a query in the blank cell below that does the following:**

Check for how many users we have 90% of their reviews (or more) in the dataset

In [0]:
# Add your query
spark.sql("""
select count(*) as userCount from datasetPercentage
where datasetPercentage>=0.9


""").show()

+---------+
userCount|
+---------+
 584949|
+---------+

### Step 12: Your Mission

Think about the number of users where you have over 90% of their data.  If you wanted to say there was a pattern in a user's data, 
and not just that we have a high percentage of the reviews they wrote, what else would you need?  If a user only wrote 
1 review, can you say much about them?

In the cell below, add a query to check for those users where you have at least 90% of their reviews, 
but they wrote 1,2,3,4, or 5 reviews.  Complete the markdown table in the subsequent markdown cell that has appropriate headers
and has rows for the number of users where you have at least 90% of their reviews and they wrote 1 - 5 reviews (your table should have 5 rows plus headers).  Each row should show the following:

* Number of reviews
* Number of users 
* Average dataset percentage (hint: in this case it should be at least 90%)

If you are unsure how to create a table in markdown, see the prior assignment.

Try getting the totals for the 5 groups in a single query.

In [0]:
# Add your query
spark.sql("""
SELECT count(user_id), ucount,avg(datasetPercentage) 
from datasetPercentage
where (ucount <=5 and datasetpercentage>=0.9)
group by ucount 
order by ucount



""").show()

+--------------+------+----------------------+
count(user_id)|ucount|avg(datasetPercentage)|
+--------------+------+----------------------+
 348539| 1| 1.0188070775436895|
 90012| 2| 1.0241578900591033|
 40583| 3| 1.02452586879564|
 23678| 4| 1.0275994594138018|
 14741| 5| 1.0288311512109116|
+--------------+------+----------------------+

### Summary of users who wrote 1 - 5 reviews and 90% or more are in the dataset:

| Reviews Written | Number of Users | Average Dataset % |
| --------------- | --------------- | ----------------- |
|     1|   348539|  1.0188070775436895|
|     2|   90012|  1.0241578900591033|
|     3|   41807|  1.024525868795464|
|     4|   24137|  1.0275994594138018|
|     5|   15247|  1.0288311512109116|